In [ ]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


In [ ]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_020823_Old");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();


In [ ]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


In [ ]:
var sessions = database.Sessions.Where(s => (s.CreationTime >= dt ));
sessions

In [ ]:
var si=sessions.Pick(1);

In [ ]:
//var sessions = database.Sessions.Where(s => (s.CreationTime >= dt && s.Name.Contains("Experimental") && s.Name.Contains("p4") &&!s.ID.ToString().Contains("c08d0fbd")&&!s.ID.ToString().Contains("2d9f7dc2")) || (s.ID.ToString().Contains("faae7fdc")&&s.Name.Contains("40x88")));
//sessions

In [ ]:
Assembler aaa = new Assembler(3);
//var si = database.Sessions.Where(s => s.ID == new Guid("012530cd-de44-4938-afc9-ba3d67933d2d")).SingleOrDefault(); //Home PC
//var si = database.Sessions.Where(s => s.ID == new Guid("51a2d86e-f457-454c-9400-5aa0322fa9c3")).SingleOrDefault(); //UNI PC
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
//IDTTimeStepInfo ts = (IDTTimeStepInfo)((TimestepProxy)database.Sessions.Pick(2).Timesteps.Last()).GetInternal();
si.ID

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=4$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 4\sqrt{\gamma}$$

In [ ]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

In [ ]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [ ]:
//&&s.Name.Contains("iFlux1")
var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
sessions=sessions.Where(s => (s.Name.Contains("iProb0")  &&s.Name.Contains("10x32")&&s.Name.Contains("agg0.2")));
sessions


# A Helper Function to obtain different plot formats

In [ ]:
using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();

public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Plot


In [ ]:

//&& s.Name.Contains("10x32")
//&& s.Name.Contains("agg0.4")
//s.CreationTime >= dt && 
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

## Residual History

In [ ]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;

foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("Res_"+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    //plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    //count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:2000,yRes:800)

## Convergence Plot Enthalpy

choose the sessions

In [ ]:
var sessions = database.Sessions.Where(s => (s.Name.Contains("iFlux1") && s.Name.Contains("agg0.4") && s.Name.Contains("iProb0")&&(s.CreationTime >= dt ) && s.SuccessfulTermination==true));
sessions

In [ ]:
var sessions2 = database.Sessions.Where(s => (s.Name.Contains("iFlux1") && s.Name.Contains("agg0.5") && s.Name.Contains("iProb0")&&(s.CreationTime >= dt ) && s.Name.Contains("40x128")));
sessions2

In [ ]:
var ses= new List<ISessionInfo>();
ses.Add(sessions.Pick(3));
ses.Add(sessions.Pick(0));
ses.Add(sessions.Pick(1));
ses.Add(sessions2.Pick(0));
//ses.Add(sessions.Pick(3));
ses

In [ ]:
int[] degs = new int[] {0,1,2,3};
int[] Cells = new int[] {5*11,5*11*2,5*11*4,5*11*8};
var errors= MultidimensionalArray.Create(4,4);


In [ ]:
sessions.Pick(2).Timesteps.Last().GetField("rho").Basis.Degree 

In [ ]:
Assembler aaa = new Assembler(3);
var timesteps = sessions.Pick(2).Timesteps;
var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == 2);
rTSIndex

In [ ]:
var plot = new Plot2Ddata();

public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
string[] STE = new string[] {"L","R"};
var C =XESFHardCodedControl.XDGBowShock_TwoLs_LSFromDB(
        optiLSDegree: 3,
        NoOfTimesteps: 20,
        dgDegreeStart: 0,
        dgDegreeEnd: 0,
        numOfCellsX: 5 * 2,
        numOfCellsY: 10 * 2
    ); 
Material mat = C.GetMaterial();
int count=0;
foreach(int p in degs){
var cells= new List<double>();
var errors= new List<double>();
for(int iSess =0;iSess < sessions.Count();iSess++){
var si = ses.Pick(iSess);
Console.WriteLine($"P={p},sess={iSess}");
    //extract the relevant field 
    var timesteps = si.Timesteps;
    var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
    if(rTSIndex !=-1){

        IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
        
            var fields = lts.Fields.ToList();
            var enthalpyError = (XDGField) lts.GetField("h").Clone();
            SpeciesId[] sIds = new SpeciesId[] {enthalpyError.Basis.Tracker.GetSpeciesId(STE[0]),enthalpyError.Basis.Tracker.GetSpeciesId(STE[1]) };
            enthalpyError.Clear();
            enthalpyError.ProjectFunctionXDG(1.0,
                    (X, U, j) => new StateVector(U, mat).Enthalpy -enthalpy_inflow, 
                    sIds,
                    lts.GetConsFields("iEuler2D"));
    //errors[p,iSess]=(enthalpyError.L2NormAllSpecies()/EEN);
    cells.Add(Math.Sqrt(enthalpyError.Basis.GridDat.iGeomCells.GetCellVolume(0)));
    errors.Add(enthalpyError.L2NormAllSpecies()/EEN);
    }
   
   
}
plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
count++;
}
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1400,yRes:900)

In [ ]:
var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
sessions=sessions.Where(s => (s.Name.Contains("iProb0")  &&s.Name.Contains("10x32")&&s.Name.Contains("agg0.3")));
sessions

In [ ]:
var si = sessions.Pick(5);
si

## Plotting the ShadowFields (for nice visualization)

In [ ]:
using BoSSS.Solution.Tecplot;
using System.IO;
Directory.GetFiles(".", "*_ShadowPlot_*").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
foreach(var timestep in si.Timesteps){
    List<DGField> flds = new List<DGField>();
    var fields = timestep.Fields.ToList();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
    texplot.PlotFields( "XESFBowShock_TwoLs_ShadowPlot_" + lts.TimeStepNumber,0.0,lts.GetShadowFields());
}